In [ ]:
%load_ext autoreload
%autoreload 2

import os
import yaml
import time
import random
import numpy as np
from tqdm import tqdm

import torch
from torch.utils import data
from tensorboardX import SummaryWriter

# Datasets
from data_loader.mnist import NoisyMNISTDataLoader

# Models
from models.autoencoder import SmallAutoEncoder
# Loss
from loss.cross_entropy_2d import CrossEntropy2d
# Metrics
from utils.metrics import MetricsComp, AverageComp

def train(config, data_set):
    # setup seeds
    torch.manual_seed(config["seed"])
    torch.cuda.manual_seed(config["seed"])
    np.random.seed(config["seed"])
    torch.cuda.empty_cache()
    
    # setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # setup dataloader
    train_data_set = data_set(split="training")
    train_data = data.DataLoader(train_data_set, batch_size=config["batch_sz"],\
                                 num_workers=config["num_workers"], shuffle=True)
    val_data = data.DataLoader(data_set(split="validation"), batch_size=config["batch_sz"],\
                               num_workers=config["num_workers"])  
    
    n_classes = 2
    
    # setup metrics
    metrics_comp = MetricsComp(n_classes)
    val_loss_avg_comp = AverageComp()

    # setup model
    model = SmallAutoEncoder().to(device)
    
    # setup optimizer
    optimizer = torch.optim.Adam(model.parameters())
        
    # setup loss
    try:
      loss_fn_weight = train_data_set.class_imbalance_weight.to(device)
    except:
      loss_fn_weight = None
    loss_fn = CrossEntropy2d(weight=loss_fn_weight)
    
    # setup tensorboard writer & checkpoint dir 
    exp_name = config.get("exp_name", "")
    uniq_name = f"{train_data_set.name}_{model.name}_{exp_name}_logs"
    writer = SummaryWriter(log_dir="checkpoints/" + uniq_name)
    
    i = 0
    # Load a saved checkpoint
    if config.get("resume_ckpoint") is not None:
        if os.path.isfile(config["resume_ckpoint"]):
            print(f"Loading checkpoint: {config['resume_ckpoint']}")
            ckpoint = torch.load(config["resume_ckpoint"])
            model.load_state_dict(ckpoint["model_state"])
            optimizer.load_state_dict(ckpoint["optimizer_state"])
            i = ckpoint["epoch"]
            print(f"Saved epoch: {i}, loss: {ckpoint['epoch_loss']}, time: {ckpoint['epoch_time']}")
        else:
            raise FileNotFoundError("Unable to load saved checkpoint!, Quitting")
            
    while i < config["epochs"]:
        i += 1
        epoch_loss = 0
        epoch_time = 0
        # Prepare for training
        model.train()
        optimizer.zero_grad()
        acc_gradients_batch = 0
        
        print(f"Starting Epoch: {i}")
        for b_i, (images, labels) in enumerate(train_data):
            start_ts = time.time()
            images = images.to(device)
            labels = labels.to(device)

            out = model(images) # [batch_sz, 2, H=28, W=28]
            acc_gradients_batch += out.shape[0]
            
            loss = loss_fn(out, labels)    
            loss.backward()
            
            # See if it is time to take a gradient step.
            if acc_gradients_batch >= 100: # gradient step every 100 samples
              optimizer.step()
              optimizer.zero_grad()
              acc_gradients_batch = 0
             
            epoch_loss += float(loss.item())
            epoch_time += time.time() - start_ts
        
        print(f"Epoch Loss: {epoch_loss}")
        writer.add_scalar(f"epoch_loss", epoch_loss, i)

        # Run through validation
        model.eval()
        with torch.no_grad():
            ctr = 0
            for _, (images_val, labels_val) in tqdm(enumerate(val_data)):
                if ctr > 10:
                    break
                ctr += 1
                images_val = images_val.to(device)
                labels_val = labels_val.to(device)               
                out = model(images_val) # [batch_sz, 2, H=28, W=28]
                val_loss = loss_fn(out, labels_val)
                val_loss_avg_comp.update(val_loss.item())
                
                _, pred = out.max(1)
                
                metrics_comp.update(label_trues=labels_val.cpu().numpy(), label_preds=pred.cpu().numpy())

        # Add validation results to tensorboard writer
        writer.add_scalar("val_loss", val_loss_avg_comp.avg, i)
        overall_scores, class_iou = metrics_comp.get_results()
        print(f"Scores after epoch {i}: {overall_scores}")
        for k, v in overall_scores.items():
            writer.add_scalar(f"val_metrics/{k}", v, i)
        for k, v in class_iou.items():
            writer.add_scalar(f"val_metrics/cls_iou_{k}", v, i)
        
        if (config["save_ckpoint"]):
          # Save the model checkpoint
          ckpoint = {"epoch":i,
                     "model_state": model.state_dict(),
                     "optimizer_state": optimizer.state_dict(),
                     "epoch_loss": epoch_loss,
                     "epoch_time": epoch_time,
                    }
          ckpoint_name = f"checkpoints/{uniq_name}_{i%3}.pkl"
          torch.save(ckpoint, ckpoint_name)
    
#TMP batch size reduced to 8->4            
config = {"exp_name": "denoise", "batch_sz": 256, "epochs": 500,\
          "seed": 3642, "num_workers": 1, "save_ckpoint": True} #"resume_ckpoint": "checkpoints/fcn32s_3.pkl", }

train(config, NoisyMNISTDataLoader)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Generating 60000 images
Generating 10000 images
Loss weight: None
Starting Epoch: 1


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4f4423c290>
Traceback (most recent call last):
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch Loss: 71.61345940828323


11it [00:00, 18.70it/s]


Scores after epoch 1: {'Overall Acc: ': 0.8765237273886828, 'FreqW Acc : ': 1681150.1773176598, 'Mean IoU : ': 967570.0}
Starting Epoch: 2


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4f4423c290>
Traceback (most recent call last):
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


Epoch Loss: 21.952831253409386


11it [00:00, 21.98it/s]


Scores after epoch 34: {'Overall Acc: ': 0.9467172744452895, 'FreqW Acc : ': 56732657.89485783, 'Mean IoU : ': 35531859.5}
Starting Epoch: 35


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4f4423c290>
Traceback (most recent call last):
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/local/google/home/ammarh/anaconda3/lib/python3.7/multiprocessing/process.py", line 138, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
